In [1]:
import os
import json
import pickle
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
# See https://radimrehurek.com/gensim/models/word2vec.html
from gensim.models import KeyedVectors

In [2]:
from tweet.tweet import Tweet

In [3]:
%%time
model = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

CPU times: user 1min 2s, sys: 1.95 s, total: 1min 3s
Wall time: 2min 9s


In [4]:
ROOT_PICKLE = '../pickle/inputs/'

In [5]:
ROOT_OUT = '../pickle/outputs/'

In [6]:
lexicon_attack = ['falso', 'mentira']

class Tweet2:
    
    tknzr = TweetTokenizer()
    
    def __init__(self, **kwargs):
        self.created_at = kwargs.get('created_at', None)
        self.id_str = kwargs.get('id_str', None)
        self.full_text = kwargs.get('full_text', None)
        self.display_text_range = kwargs.get('display_text_range', None)
        self.in_reply_to_status_id_str = kwargs.get('in_reply_to_status_id_str', None)
        self.screen_name = kwargs.get('screen_name')
        
        self.replies = None
        self.tokens = None
        self.stance = None
        
        self.get_text()
        self.tokenizer()
    
    def get_text(self):
        self.full_text = self.full_text[self.display_text_range[0]:]
    
    def tokenizer(self):
        self.tokens = self.tknzr.tokenize(self.full_text)
    
    def set_stance(self, stance):
        # attack, or support
        self.stance = stance
    
    def is_attack(self, lexicon):
        attack = False
        for token in self.tokens:
            if token in lexicon:
                self.set_stance('attack')
                attack = True
                break
        return attack
    
    def is_controversial(self, lexicon):
        # solo chequeo el segundo nivel
        attacks = 0
        for reply in self.replies:
            if reply.is_attack(lexicon):
                attacks += 1
        return attacks >= 2

In [10]:
def uno(file):
    tweets = defaultdict()
    root = None
    
    with open(os.path.join(ROOT_PICKLE, file), "r") as fd:
        for line in fd:
            jn = json.loads(line)
            
            created_at = jn['created_at']
            id_str = jn['id_str']
            full_text = jn['full_text']
            display_text_range = jn['display_text_range']
            in_reply_to_status_id_str = jn['in_reply_to_status_id_str']
            screen_name = jn['user']['screen_name']
            
            if not in_reply_to_status_id_str:
                root = id_str
            
            tweets[id_str] = Tweet(created_at=created_at,
                                   id_str=id_str,
                                   full_text=full_text,
                                   display_text_range=display_text_range,
                                   in_reply_to_status_id_str=in_reply_to_status_id_str,
                                   screen_name=screen_name
                                  )
    return tweets, root

In [11]:
def dos(tweets, root):
    r = defaultdict(list)
    
    for id_st, tweet in tweets.items():
        if tweet.in_reply_to_status_id_str:
            r[tweet.in_reply_to_status_id_str].append(tweet)
    
    for k, v in r.items():
        tweets[k].replies = v
    
    return tweets[root]

In [12]:
tweets, root = uno('perfil.status.1191780164549697536.json')

In [13]:
tree = dos(tweets, root)

**Primer resultado**

In [14]:
tree.is_controversial(lexicon_attack)

True

In [15]:
def tageo(tree_dict, lexicon):
    for _, tweet in tree_dict.items():
        tweet.is_attack(lexicon)

In [16]:
tageo(tweets, lexicon_attack)

In [17]:
tree = dos(tweets, root)

In [18]:
def pri(tree):
    for reply in tree.replies:
        if reply.stance:
            print(reply.full_text)
        if reply.replies:
            pri(reply)

In [19]:
pri(tree)

Es falso.
Saben que es mentira, saben que son unos delincuentes, e igual lo difunden. En la dictadura se los llamo socios civiles . Ustedes SON CÓMPLICES


In [20]:
with open(os.path.join(ROOT_OUT, 'example1iter.pkl'), 'wb') as fd:
    pickle.dump(tree, fd)

**Segundo resultado**

In [22]:
lexicon_attack_iter_2 = lexicon_attack + [word for word, d in model.most_similar(positive=lexicon_attack)]

In [23]:
lexicon_attack_iter_2

['falso',
 'mentira',
 'falsa',
 'mentirilla',
 'verdad',
 'fraudulencia',
 'engaño',
 'infundio',
 'mentiras',
 'andrómina',
 'patraña',
 'encubriéndola']

In [24]:
tageo(tweets, lexicon_attack_iter_2)

In [25]:
pri(tree)

Es falso.
Que se vaya macri y que deje de mentir tantas mentiras puede perjudicar el país
mentiras hasta último día mienten hdmp
La verdad dejan deuda fmi
Sera cierto esto?...mmmm...la verdad....tengo mis dudasss..
Son increíbles no dejan de mentir nunca.
No han tenido un solo acierto en lo económico, político, social,cultural en ninguno,son horribles de verdad.
Por favor váyanse a disfrutar de sus mentiras y dólares acumulados y no hagan más daño!!!!
Saben que es mentira, saben que son unos delincuentes, e igual lo difunden. En la dictadura se los llamo socios civiles . Ustedes SON CÓMPLICES
Más mentiras macrisis
Ibarra es un tipo serio no puede decir esto! Tenemos que pagar intereses en el corto plazo que antes no, y está la deuda con el FMI : como te queda la cuenta neta? Digamos la verdad total no a medias
Manga de ladrones digan la verdad!! Que hicieron con la guita del FMI ...se ROBARON UN FMI ENTERO CARADURA!
basta de mentiras perfil............no jodan mas


In [27]:
with open(os.path.join(ROOT_OUT, 'example2iter.pkl'), 'wb') as fd:
    pickle.dump(tree, fd)

IDEA FRANCO:

1- AGRANDO LEXICON A PARTIR DEL MODELO DE CRISTIAN

2- CALCULO UN UMBRAL DEL MODELO DE CRISTIAN

3- PARA CADA (PALABRAL, TOKENT) DEL LEXICON X TWEET, SI DISTANCIA(PALABRAL, TOKENT) < UMBRAL ENTONCES LEXICON + TOKENT 

**API TWITTER**

In [ ]:
import tweepy

In [ ]:
CONSUMER_KEY = 'tRe3ZBhnXDPBEKyTi4ifvPTC7' 
CONSUMER_SECRET = '0dmQlSimyEmaOPiDoFZTF0wYjO0LSzSdWmCP7afZkNHZF94Af5'
ACCESS_TOKEN = '421837149-YVewRWLSu8xyYNU1H4NtELUejGB8lzvT4KCfallG'
ACCESS_SECRET = 'mFsKYpvJaWvZGHRoy874Z4IF09CRP69srlVj2yjN6mH5L'

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
search = tweepy.Cursor(api.search, q="to:FedeFavre", sinceId = 1188249861721919489).items()